In [1]:
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
from mlxtend.plotting import plot_decision_regions
import sklearn
from torch.nn.functional import one_hot

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"

In [3]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight','Acceleration','Model Year','Origin']

In [4]:
# df = pd.read_csv(url,names=column_names,na_values="?",comment="\t",sep=" ",skipinitialspace=True)

In [5]:
df = pd.read_csv("/Users/abhishekbose/Downloads/auto-mpg.data",names=column_names,na_values="?",comment="\t",sep=" ",skipinitialspace=True)

In [6]:
df.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1


In [7]:
df.dropna(inplace=True)
df.reset_index(drop=True)

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1
...,...,...,...,...,...,...,...,...
387,27.0,4,140.0,86.0,2790.0,15.6,82,1
388,44.0,4,97.0,52.0,2130.0,24.6,82,2
389,32.0,4,135.0,84.0,2295.0,11.6,82,1
390,28.0,4,120.0,79.0,2625.0,18.6,82,1


In [8]:
df.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1


In [9]:
#train/test split
df_train, df_test = sklearn.model_selection.train_test_split(df,train_size=0.8,random_state=1)

In [10]:
train_stats = df_train.describe().transpose()

In [11]:
train_stats

,count,mean,std,min,25%,50%,75%,max
MPG,313.0,23.404153,7.666909,9.0,17.5,23.0,29.0,46.6
Cylinders,313.0,5.402556,1.701506,3.0,4.0,4.0,8.0,8.0
Displacement,313.0,189.512780,102.675646,68.0,104.0,140.0,260.0,455.0
Horsepower,313.0,102.929712,37.919046,46.0,75.0,92.0,120.0,230.0
Weight,313.0,2961.198083,848.602146,1613.0,2219.0,2755.0,3574.0,5140.0
Acceleration,313.0,15.704473,2.725399,8.5,14.0,15.5,17.3,24.8
Model Year,313.0,75.929712,3.675305,70.0,73.0,76.0,79.0,82.0
Origin,313.0,1.591054,0.807923,1.0,1.0,1.0,2.0,3.0


In [12]:
numeric_column_names = ['Cylinders','Displacement','Horsepower','Weight','Acceleration']

In [13]:
df_train_norm, df_test_norm = df_train.copy(), df_test.copy()

In [14]:
for col_name in numeric_column_names:
    mean = train_stats.loc[col_name,'mean']
    std = train_stats.loc[col_name,'std']
    df_train_norm.loc[:,col_name] = (df_train_norm.loc[:,col_name] - mean)/std
    df_test_norm.loc[:,col_name] = (df_test_norm.loc[:,col_name] - mean)/std

In [15]:
df_train_norm.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
205,28.0,-0.824303,-0.901020,-0.736562,-0.950031,0.255202,76,3
257,19.4,0.351127,0.413800,-0.340982,0.293190,0.548737,78,1
73,13.0,1.526556,1.144256,0.713897,1.339617,-0.625403,72,1
237,30.5,-0.824303,-0.891280,-1.053025,-1.072585,0.475353,77,1
38,14.0,1.526556,1.563051,1.636916,1.470420,-1.359240,71,1


In [16]:
'''
year bucket = {
    0 if year < 73
    1 if 73<= year < 76
    2 if 76<= year < 79
    3 if year>= 79
}
'''
boundaries = torch.tensor([73,76,79])
v = torch.tensor(df_train_norm['Model Year'].values)
df_train_norm['Model Year Bucketed'] = torch.bucketize(
    v, boundaries, right=True
) 

In [17]:
df_train_norm.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin,Model Year Bucketed
338,27.2,-0.824303,-0.530922,-0.499214,-0.555264,-0.001641,81,1,3
260,18.6,0.351127,0.345625,0.186457,0.776338,1.099115,78,1,2
141,29.0,-0.824303,-0.891280,-0.525586,-0.874613,0.291894,74,2,1
312,37.2,-0.824303,-1.008153,-1.000281,-1.110294,0.255202,80,3,3
353,33.0,-0.824303,-0.823104,-0.762934,-0.908786,-0.552019,81,2,3


In [18]:
v = torch.tensor(df_test_norm['Model Year'].values)
df_test_norm['Model Year Bucketed'] = torch.bucketize(
    v, boundaries,right=True
)

In [19]:
numeric_column_names

['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration']

In [20]:
numeric_column_names.append("Model Year Bucketed")

In [21]:
numeric_column_names

['Cylinders',
 'Displacement',
 'Horsepower',
 'Weight',
 'Acceleration',
 'Model Year Bucketed']

In [22]:
total_origin = len(set(df_train_norm['Origin']))

In [23]:
origin_encoded = one_hot(torch.from_numpy(df_train_norm['Origin'].values) % total_origin)

In [24]:
x_train_numeric = torch.tensor(df_train_norm[numeric_column_names].values)

In [25]:
x_train_numeric.shape

torch.Size([313, 6])

In [26]:
origin_encoded

tensor([[0, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 

In [27]:
x_train = torch.cat([x_train_numeric,origin_encoded],1).float()

In [28]:
origin_encoded = one_hot(torch.from_numpy(df_test_norm['Origin'].values) % total_origin)

In [29]:
x_test_numeric = torch.tensor(df_test_norm[numeric_column_names].values)
x_test = torch.cat([x_test_numeric,origin_encoded],1).float()

In [30]:
#ground truth value is MPG
y_train = torch.tensor(df_train_norm['MPG'].values).float()
y_test = torch.tensor(df_test_norm['MPG'].values).float()

In [31]:
x_train.shape

torch.Size([313, 9])

In [32]:
train_ds = TensorDataset(x_train, y_train)
batch_size = 8
torch.manual_seed(1)
train_dl = DataLoader(train_ds,batch_size=batch_size,shuffle=True)

In [33]:
hidden_units = [8,4]
input_size = x_train.shape[1]
all_layers = []
for hidden_unit in hidden_units:
    layer = nn.Linear(input_size,hidden_unit)
    all_layers.append(layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit

In [34]:
all_layers

[Linear(in_features=9, out_features=8, bias=True),
 ReLU(),
 Linear(in_features=8, out_features=4, bias=True),
 ReLU()]

In [35]:
all_layers.append(nn.Linear(hidden_units[-1],1))

In [36]:
all_layers

[Linear(in_features=9, out_features=8, bias=True),
 ReLU(),
 Linear(in_features=8, out_features=4, bias=True),
 ReLU(),
 Linear(in_features=4, out_features=1, bias=True)]

In [37]:
model = nn.Sequential(*all_layers)

In [38]:
model

Sequential(
  (0): Linear(in_features=9, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): ReLU()
  (4): Linear(in_features=4, out_features=1, bias=True)
)

In [39]:
numeric_column_names

['Cylinders',
 'Displacement',
 'Horsepower',
 'Weight',
 'Acceleration',
 'Model Year Bucketed']

In [40]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.001)

In [41]:
torch.manual_seed(1)
num_epochs = 500
log_epochs = 20

In [42]:
for epoch in range(num_epochs):
    loss_hist_train = 0
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)[:,0]
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss_hist_train += loss.item()
    if epoch % log_epochs == 0:
        print(f'Epoch {epoch} Loss {loss_hist_train/len(train_dl):.4f}')

Epoch 0 Loss 542.0038
Epoch 20 Loss 7.8073
Epoch 40 Loss 7.6701
Epoch 60 Loss 6.9267
Epoch 80 Loss 7.2349
Epoch 100 Loss 7.4779
Epoch 120 Loss 6.3839
Epoch 140 Loss 6.3135
Epoch 160 Loss 6.2078
Epoch 180 Loss 6.0783
Epoch 200 Loss 6.6420
Epoch 220 Loss 6.2648
Epoch 240 Loss 6.8009
Epoch 260 Loss 6.0452
Epoch 280 Loss 5.7043
Epoch 300 Loss 5.8074
Epoch 320 Loss 5.1616
Epoch 340 Loss 5.8106
Epoch 360 Loss 5.3452
Epoch 380 Loss 5.0873
Epoch 400 Loss 5.5356
Epoch 420 Loss 5.4240
Epoch 440 Loss 5.1499
Epoch 460 Loss 5.1471
Epoch 480 Loss 6.4913


In [43]:
with torch.no_grad():
    pred = model(x_test.float())[:,0]
    loss = loss_fn(pred, y_test)
    print(f'Test MSE: {loss.item():.4f}')
    print(f'Test MAE: {nn.L1Loss()(pred,y_test).item():.4f}')

Test MSE: 13.5185
Test MAE: 2.4166
